In [ ]:
from servoserial import ServoSerial
import time
servo_device = ServoSerial()

In [ ]:
servo_device.Servo_serial_double_control(1, 2100, 2, 2048)
time.sleep(0.1)

In [ ]:
def yes() : 
    servo_device.Servo_serial_control(2, 1600)
    time.sleep(0.3)
    servo_device.Servo_serial_control(2, 2048)
    time.sleep(0.3)

In [ ]:
def no() : 
    servo_device.Servo_serial_control(1, 1700)
    time.sleep(0.5)
    servo_device.Servo_serial_control(1, 2500)
    time.sleep(0.5)
    servo_device.Servo_serial_control(1, 2100)
    time.sleep(0.3)

In [ ]:
from jetbot import Robot
import time
robot = Robot()

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
from jetbot import Camera

#camera = Camera.instance(width=224, height=224, fps=10)
camera = Camera.instance(width=300, height=300)

In [ ]:
detections = model(camera.value)

print(detections)

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

In [ ]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np


mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    # Image zoomed to 224,224 versus 224,244 obstacle avoidance model
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
def btch_iou(a, b, epsilon=1e-5) :
    
    # a : bbox of target
    # b : bbox of other
    
    # 겹치는 부분의 좌표
    x1 = np.maximum(a[0], b[:,0])
    y1 = np.maximum(a[1], b[:,1])
    x2 = np.minimum(a[2], b[:,2])
    y2 = np.minimum(a[3], b[:,3])
    
    #겹치는 부분 길이
    iou_width = (x2 - x1)
    iou_height = (y2 - y1)
    
    # 겹치지 않을 경우
    iou_width[iou_width < 0] = 0
    iou_height[iou_height < 0] = 0
    
    # 겹치는 넓이
    area_overlap = iou_width * iou_height
    
    # 전체 넓이
    area_a = ( a[2] - a[0] ) * ( a[3] - a[1] )
    area_b = ( b[:,2] - b[:,0] ) * ( b[:,3] - b[:,1] )
    area_combined = area_a + area_b - area_overlap
    
    #IOU
    iou = area_overlap / (area_combined + epsilon)
    print(iou)
    
    return iou

In [ ]:
from jetbot import bgr8_to_jpeg


image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')

display(widgets.VBox([
    widgets.HBox([image_widget]),
    label_widget,
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection


def execute(change, target_person):
    image = change['new']
 
    prev_match_det = target_person # 이전 target 값 받기

    # compute all detected objects
    detections = model(image)
    b_bbox = []
    
    print()
    print()
    print('현재 bbox')
    
    
    # 전체 object 파란 박스로 표시하기
    for det_bx in detections[0]:
        bbox = det_bx['bbox']
        label = det_bx['label']
        confidence = det_bx['confidence']
        cv2.putText(image, '%s, %.4f' %(label,confidence), (int(300 * bbox[0]), int(300 * bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
        b_bbox.append(det_bx['bbox'])
    
    
    # 새 object box 정보 저장
    if len(b_bbox) == 0:
        b_bbox = np.array([[0.0, 0.0, 0.0, 0.0]])
    elif len(b_bbox) == 1:
        b_bbox = np.array(b_bbox).reshape((1,4))
    else:
        b_bbox = np.array(b_bbox)
    
    print('b_bbox : ')
    print(b_bbox)
    
    
    # 사람이라면 목표 타겟으로 설정하기
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    
    # 이전 타겟 정보가 없을 때 det 결정
    if prev_match_det == None:
        print('시작!')
        det = closest_detection(matching_detections)  # 현재 frame에서 타겟 찾기 (목표 타겟 중 가운데에 가장 가까운 것)
        print('pmd==none : ')
        print(det)
        
    # 이전 타겟 정보가 있을 때 det 결정
    else:
        print()
        prev_a_bbox = prev_match_det['bbox']  # 이전 detection의 target bounding box 정보 가져오기
        c = np.argmax(btch_iou(prev_a_bbox, b_bbox))  # 이전 target bbox와 현재 전체 bbox 중에서 iou 최대 계산
        
        if np.sum(b_bbox[0]) != 0 :  # 최대 iou 만드는 box가 존재
            print('새 box 포착')
            d = detections[0][c]  # iou 최대인 개체를 target으로 지정
            
            if d['label'] == int(label_widget.value): # iou 최대 개체가 사람
                print('새 box가 사람')
                det = d  # 새 target으로 변경
            else:  # iou 최대 개체가 사람이 아님
                print('새 box가 사람 아님')
                det = prev_match_det # 이전 target 유지
                
            print('pmd ok / d : ')
            print(d)               
        
        elif np.sum(b_bbox) == 0 : # 새 box가 없음
            print('새 box가 없음')
            det = prev_match_det # 이전 target 유지
            
        print('pmd ok / det : ')
        print(det)
    
    
    # 결정된 det를 초록색 박스로 표시
    if det is not None:
        bbox = det['bbox']
        label = det['label']
        confidence = det['confidence']
        cv2.putText(image, '%s, %.4f' %(label,confidence), (int(300 * bbox[0]), int(300 * bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (0, 255, 0), 4)
        a_bbox = np.array(det['bbox'])
        print('a_bbox : ')
        print(a_bbox)

    
    
    if det is None:
        #det = prev_match_det
        time.sleep(0.5)
        if det is None:
            no()
    
    else:
        center = detection_center(det)


        if label_widget.value == 1:
            
            servo_device.Servo_serial_double_control(1, 2100, 2, 2150)
            time.sleep(0.1)
            
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            
            # bbox 즁심을 image 중심으로 맞추기
            if center[0] > 0.1 :
                robot.right(0.55)
            elif center[0] < -0.1 :
                robot.left(0.55)
            else :
                # bbox 작아지면 따라가기
                if w*h < 0.20 :
                    robot.forward(0.55)
                else :
                    robot.stop()
            
            
            if np.max(btch_iou(a_bbox, b_bbox)) > 0.1 :  # detections 중에서 최대 제외 iou가 일정 이상인 box가 존재하면 :
                inst = np.argmax(btch_iou(a_bbox, b_bbox))
                max_box = detections[0][inst]
                max_box_center = detection_center(max_box)

                if max_box['label'] == int(label_widget.value) : 
                    #무시
                    time.sleep(0.1)
                else :
                    #if det['confidence'] < 0.6 :
                    if center[0] < max_box_center[0] : # 겹치는 box center거 det center보다 오른쪽 :
                            #왼쪽으로 갔다가 다시 오른쪽
                        robot.set_motors(0.3, 0.7)
                        time.sleep(0.5)
                        robot.set_motors(0.5, 0.6)
                        time.sleep(0.5)
                        robot.set_motors(0.7, 0.3)
                        time.sleep(0.5)
                        robot.set_motors(0.6, 0.6)
                        time.sleep(0.5)
                        robot.stop()
                    elif center[0] > max_box_center[0] : # 겹치는 box center가 det center보다 왼쪽 :
                            #오른쪽으로 갔다가 다시 왼쪽
                        robot.set_motors(0.3, 0.7)
                        time.sleep(0.5)
                        robot.set_motors(0.5, 0.6)
                        time.sleep(0.5)
                        robot.set_motors(0.7, 0.3)
                        time.sleep(0.5)
                        robot.set_motors(0.6, 0.6)
                        time.sleep(0.5)
                        robot.stop()
                    
                    else :
                        #무시
                        time.sleep(0.1)
        
        else :
            time.sleep(0.5)
            no()


    
        if label_widget.value != 1:
            #det = prev_match_det
            time.sleep(0.5)
            no()


    # Update image display to widget
    image_widget.value = bgr8_to_jpeg(image)
    
#    prev_match_det = det

    return det

person = None
person = execute({'new': camera.value}, person)

In [ ]:
#camera.unobserve_all()
#camera.observe(execute, names='value')

import threading
import inspect
import ctypes
''' definition of the close thread function'''
def _async_raise(tid, exctype):
  """raises the exception, performs cleanup if needed"""
  tid = ctypes.c_long(tid)
  if not inspect.isclass(exctype):
    exctype = type(exctype)
  res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
  if res == 0:
    raise ValueError("invalid thread id")
  elif res != 1:
    # """if it returns a number greater than one, you're in trouble,
    # and you should call it again with exc=NULL to revert the effect"""
    ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
    raise SystemError("PyThreadState_SetAsyncExc failed")
def stop_thread(thread):
  _async_raise(thread.ident, SystemExit)
'''The function of thread 1, which continuously calls the detection function'''
def test():
    person = None
    while True:
        person = execute({'new': camera.value}, person)        

thread1 = threading.Thread(target=test)
thread1.setDaemon(False)
thread1.start()


In [ ]:
import time
stop_thread(thread1)
camera.unobserve_all()
time.sleep(1.0)
robot.stop()

In [ ]:
matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]

a = closest_detection(matching_detections)
b = detections

a_bbox = np.array(a['bbox'])
b_bbox = []

for det in detections[0]:
    print(len(b_bbox))
    b_bbox.append(det['bbox'])
    print(len(b_bbox))

bbx_size = len(b_bbox)
    
b_bbox = np.array(b_bbox)

if bbx_size == 0:
    b_bbox = []
elif bbx_size == 1:
    np.expand_dims(b_bbox, axis=0)

print(detections)
print()
print(closest_detection(matching_detections))
print()
print(a_bbox)
print()
print(b_bbox)
print()

btch_iou(a_bbox, b_bbox)

c = np.argmax(btch_iou(a_bbox, b_bbox))

#print(closest_detection(matching_detections))
print(detections[0][c])
#print(d)